# Relative Pose Calibration

In [24]:
import cv2
import numpy as np
import os

In [25]:
# Read the images
base_dir = './images'
rgb_folder1 = f'{base_dir}/calib/camera1_rgb'
rgb_folder2 = f'{base_dir}/calib/camera2_rgb'
depth_folder1 = f'{base_dir}/calib/camera1_depth'
depth_folder2 = f'{base_dir}/calib/camera2_depth'

# Get list of image paths for camera 1 and camera 2
rgb_paths1 = [os.path.join(rgb_folder1, f) for f in os.listdir(rgb_folder1) if f.endswith('.png')]
rgb_paths2 = [os.path.join(rgb_folder2, f) for f in os.listdir(rgb_folder1) if f.endswith('.png')]
depth_paths1 = [os.path.join(depth_folder1, f) for f in os.listdir(depth_folder1) if f.endswith('.npy')]
depth_paths2 = [os.path.join(depth_folder2, f) for f in os.listdir(depth_folder2) if f.endswith('.npy')]

In [26]:
# Load the intrinsic and extrinsic parameters
camera1_params = np.load(f'{base_dir}/calib/camera1_params.npz')
camera2_params = np.load(f'{base_dir}/calib/camera2_params.npz')
intrinsic_matrix1 = camera1_params['color_intrinsics']
intrinsic_matrix2 = camera2_params['color_intrinsics']
dist_coeffs1 = camera1_params['color_distortion']
dist_coeffs2 = camera2_params['color_distortion']
depth_scale1 = camera1_params['depth_scale']
depth_scale2 = camera2_params['depth_scale']

print('Intrinsic matrix 1:', intrinsic_matrix1)
print('Intrinsic matrix 2:', intrinsic_matrix2)
print('Distortion coefficients 1:', dist_coeffs1)
print('Distortion coefficients 2:', dist_coeffs2)
print('Depth scale 1:', depth_scale1)
print('Depth scale 2:', depth_scale2)

Intrinsic matrix 1: [[1.36377332e+03 0.00000000e+00 9.56298462e+02]
 [0.00000000e+00 1.36157141e+03 5.46581238e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Intrinsic matrix 2: [[1.36576904e+03 0.00000000e+00 9.50094910e+02]
 [0.00000000e+00 1.36530176e+03 5.55031494e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients 1: [0 0 0 0 0]
Distortion coefficients 2: [0 0 0 0 0]
Depth scale 1: 0.0010000000474974513
Depth scale 2: 0.0010000000474974513


In [27]:
# Define Charuco board parameters
charuco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
board = cv2.aruco.CharucoBoard((5, 5), 0.08, 0.06, charuco_dict)

# Function to process an image and estimate pose
def estimate_pose(image_path, camera_matrix, dist_coeffs):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = cv2.aruco.detectMarkers(gray, charuco_dict)
    
    if len(corners) > 0:
        ret, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(corners, ids, gray, board)
        if charuco_ids is not None and len(charuco_corners) > 3:
            valid, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, camera_matrix, dist_coeffs, None, None)
            if valid:
                return rvec, tvec
    return None, None

# Calculate relative pose between two cameras
def calculate_relative_pose(rvec1, tvec1, rvec2, tvec2):
    # Convert rotation vectors to rotation matrices
    R_cam1_to_world, _ = cv2.Rodrigues(rvec1)
    R_cam2_to_world, _ = cv2.Rodrigues(rvec2)
    
    # Calculate rotation matrix from camera 1 to camera 2
    R_cam1_to_cam2 = np.dot(R_cam2_to_world, R_cam1_to_world.T)
    
    # Calculate translation vector from camera 1 to camera 2
    t_cam1_to_cam2 = tvec2 - np.dot(R_cam1_to_cam2, tvec1)
    
    return R_cam1_to_cam2, t_cam1_to_cam2

# List to store relative poses
relative_poses = []

# Loop through all image pairs and calculate relative pose
for image_path1, image_path2 in zip(rgb_paths1, rgb_paths2):
    # Estimate pose for camera 1
    rvec1, tvec1 = estimate_pose(image_path1, intrinsic_matrix1, dist_coeffs1)
    
    # Estimate pose for camera 2
    rvec2, tvec2 = estimate_pose(image_path2, intrinsic_matrix2, dist_coeffs2)
    
    if rvec1 is not None and rvec2 is not None:
        # Calculate relative pose
        relative_pose = calculate_relative_pose(rvec1, tvec1, rvec2, tvec2)
        relative_poses.append(relative_pose)

In [28]:
# Calculate average relative pose
R_avg = np.zeros((3, 3))
t_avg = np.zeros((3, 1))
for R, t in relative_poses:
    R_avg += R
    t_avg += t

# make sure R_avg is a proper rotation matrix
R_avg /= len(relative_poses)
U, S, Vt = np.linalg.svd(R_avg)
R_avg = np.dot(U, Vt)

t_avg /= len(relative_poses)

# Print average relative pose
print('Average relative rotation:')
print(R_avg)
print('Average relative translation:')
print(t_avg)

Average relative rotation:
[[-0.98752715 -0.04611448  0.1505443 ]
 [ 0.12623187 -0.80335989  0.58196083]
 [ 0.09410443  0.5937056   0.79916082]]
Average relative translation:
[[-0.00960099]
 [-0.63541039]
 [ 0.74765755]]


In [29]:
# calculate the mean error
tot_error = 0
for R, t in relative_poses:
    error = np.linalg.norm(R - R_avg) + np.linalg.norm(t - t_avg)
    print('Error:', error)
    tot_error += error
mean_error = tot_error / len(relative_poses)
print('Mean error:', mean_error)

Error: 0.08818418242638391
Error: 0.010917324224961451
Error: 0.07693433539884519
Error: 0.04076680274855825
Error: 0.0851724390459744
Error: 0.09222404627766553
Error: 0.0767770782053251
Mean error: 0.06728231547538768


In [30]:
import open3d as o3d
import numpy as np

# Define the original rotation matrix and translation vector
R_original = np.eye(3)  # Identity matrix? for no rotation
t_original = np.zeros((3, 1))  # Zero translation

# Define the relative rotation matrix and translation vector
R_relative = R_avg.copy()
t_relative = t_avg.copy()

# Create a 4x4 transformation matrix for the original pose
T_original = np.eye(4)
T_original[:3, :3] = R_original
T_original[:3, 3] = t_original.flatten()

# Create a 4x4 transformation matrix for the relative pose
T_relative = np.eye(4)
T_relative[:3, :3] = R_relative
T_relative[:3, 3] = t_relative.flatten()

# Combine the transformations to get the transformed pose
T_transformed = np.dot(T_original, T_relative)

# Create coordinate frames for the original and transformed poses
mesh_frame_original = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6)
mesh_frame_transformed = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6)

# Transform the coordinate frames
mesh_frame_original.transform(T_original)
mesh_frame_transformed.transform(T_transformed)

# Visualize both poses
o3d.visualization.draw_geometries([mesh_frame_original, mesh_frame_transformed])

# Simple accumulation of the pointcloud

In [31]:
# Read the images
base_dir = './images'
rgb_folder1 = f'{base_dir}/test/camera1_rgb'
rgb_folder2 = f'{base_dir}/test/camera2_rgb'
depth_folder1 = f'{base_dir}/test/camera1_depth'
depth_folder2 = f'{base_dir}/test/camera2_depth'

# Get list of image paths for camera 1 and camera 2
rgb_paths1 = [os.path.join(rgb_folder1, f) for f in os.listdir(rgb_folder1) if f.endswith('.png')]
rgb_paths2 = [os.path.join(rgb_folder2, f) for f in os.listdir(rgb_folder1) if f.endswith('.png')]
depth_paths1 = [os.path.join(depth_folder1, f) for f in os.listdir(depth_folder1) if f.endswith('.npy')]
depth_paths2 = [os.path.join(depth_folder2, f) for f in os.listdir(depth_folder2) if f.endswith('.npy')]

In [32]:
camera1_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera1_intrinsic.set_intrinsics(intrinsic_matrix1.shape[1], intrinsic_matrix1.shape[0], intrinsic_matrix1[0, 0], intrinsic_matrix1[1, 1], intrinsic_matrix1[0, 2], intrinsic_matrix1[1, 2])

camera2_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera2_intrinsic.set_intrinsics(intrinsic_matrix2.shape[1], intrinsic_matrix2.shape[0], intrinsic_matrix2[0, 0], intrinsic_matrix2[1, 1], intrinsic_matrix2[0, 2], intrinsic_matrix2[1, 2])

extrinsics2 = np.eye(4)
extrinsics1 = np.eye(4)
extrinsics1[:3, :3] = R_avg
extrinsics1[:3, 3] = t_avg.flatten()

In [33]:
rgb_img1 = cv2.imread(rgb_paths1[0])
depth_img1 = np.load(depth_paths1[0])
rgb_img2 = cv2.imread(rgb_paths2[0])
depth_img2 = np.load(depth_paths2[0])
# Create RGBD image
rgbd_image1 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    o3d.geometry.Image(rgb_img1),
    o3d.geometry.Image(depth_img1),
    depth_scale=1.0/depth_scale1,
    # depth_trunc=3.0
)

rgbd_image2 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    o3d.geometry.Image(rgb_img2),
    o3d.geometry.Image(depth_img2),
    depth_scale=1.0/depth_scale2,
    # depth_trunc=3.0
)

pcd1 = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image1,
    camera1_intrinsic
)

pcd2 = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image2,
    camera2_intrinsic
)

# transform point cloud
pcd1.transform(extrinsics1)

PointCloud with 1729770 points.

In [34]:
# export the pointcloud as .ply
o3d.io.write_point_cloud("output_point_cloud.ply", pcd2)

True

In [35]:
# pcd1 to blue
pcd1.paint_uniform_color([0, 0, 1])

# Visualize
o3d.visualization.draw_geometries([pcd1,pcd2])